In [1]:
# Ollama 모델 로드 및 테스트
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="gemma:7b-instruct", temperature=0)

response = model.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")

In [3]:
response

AIMessage(content='겨울철에 내한성이 강한 나무에는 다음과 같은 나무가 포함됩니다.\n\n**주요 내한성 나무:**\n\n* **소나무:** 전국에 걸쳐 분포되어 있으며, 겨울철에는 강한 내한성을 보유합니다.\n* **참나무:** 동쪽의 해변가에서 많이 볼 수 있는 나무로, 겨울철에는 강한 내한성을 보유합니다.\n* **사나버:** 동쪽의 해변가에서 많이 볼 수 있는 나무로, 겨울철에는 강한 내한성을 보유합니다.\n* **오크:** 북동부의 산맥에서 많이 볼 수 있는 나무로, 겨울철에는 강한 내한성을 보유합니다.\n* **느티:** 북부의 산맥에서 많이 볼 수 있는 나무로, 겨울철에는 강한 내한성을 보유합니다.\n\n**겨울철에 내한성이 약한 나무:**\n\n* **동백:** 겨울철에는 내한성이 약하여 주의를 요구합니다.\n* **수나:** 겨울철에는 내한성이 약하여 주의를 요구합니다.\n* **사자:** 겨울철에는 내한성이 약하여 주의를 요구합니다.\n* **주목:** 겨울철에는 내한성이 약하여 주의를 요구합니다.\n\n**겨울철에 내한성이 강한 나무를 선택하려면 다음 요소를 고려하세요:**\n\n* **위치:** 겨울철에 강한 내한성을 보유한 나무는 해풍이나 눈보온 지역에 잘 적응합니다.\n* **종:** 겨울철에 강한 내한성을 보유한 나무 종은 다양하지만, 소나무, 참나무, 사나버 등이 가장 강한 내한성을 보유합니다.\n* **성장:** 겨울철에 강한 내한성을 보유한 나무는 성장이 잘 이루어집니다.', response_metadata={'model': 'gemma:7b-instruct', 'created_at': '2024-03-30T05:07:17.867944Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 154934861100, 'load_duration': 10243082700, 'prompt_eval_count': 29, 'prom

In [6]:
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("경남교육청_수목관리_업무매뉴얼.pdf", extract_images=False)
pages = loader.load()
pages[0].page_content

'경남교육 2022 - 250\n수\n목 관리 업무매뉴얼\n수목 관리 \n업무매뉴얼 쾌적한 학교 공간 재구성을 위한\n 쾌적한 학교 공간 재구성을 위한\n수목 관리\n업무매뉴얼҃թҮਭ'

In [7]:
# 문장 임베딩 및 벡터 저장소 생성
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

docs = text_splitter.split_documents(pages)

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)


c:\Users\tsdata\AppData\Local\pypoetry\Cache\virtualenvs\ollama-pdf-rag-j5v6ObUr-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tsdata\AppData\Local\pypoetry\Cache\virtualenvs\ollama-pdf-rag-j5v6ObUr-py3.11\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tsdata\.cache\huggingface\hub\models--jhgan--ko-sroberta-nli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, y

In [8]:
# 검색 쿼리
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

docs = retriever.get_relevant_documents(query)
print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

5
나) 수종별 내한성에 따른 전정 
  추위를 잘 견디어 내는 나무의 성질을 내한성이라고 한다. 생장 중인 잎이나 가는 가지 등에서 일어나는 상해(霜害)에 대한 저항성을 포함하는 경우와, 휴면 중인 가지나 엄동기
{'page': 139, 'source': '경남교육청_수목관리_업무매뉴얼.pdf'}


In [9]:
print(docs[1].page_content)
print(docs[1].metadata)

(1) 내한성은 동절기 중 극히 낮은 온도에 견디는 능력이다. 
(가) 내한성이 강한 수종은 한대림에서 자라는 수종으로 자작나무, 오리나무, 사
시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당된다. 
(나) 내한성이 약한 수종은 삼나무, 편백, 금송, 히말라야시다, 배롱나무, 파라칸
타, 동백나무, 후박나무, 먼나무 등 주로 남부 지역에서 자라는 수종과 자목련, 사철나무, 가이즈까향나무, 능소화, 벽오동, 오동나무 등이다. 
(다) 내한성이 약한 수종은 수간을 볏짚이나 새끼 끈으로 싸 주고, 상열을 막기
{'page': 110, 'source': '경남교육청_수목관리_업무매뉴얼.pdf'}


In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)


'겨울철에 내한성이 강한 나무에는 자작나무, 오리나무, 사시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당됩니다.'

In [11]:
query = "겨울철에 추위에 약한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)

'겨울철에 추위에 약한 나무에는 잎이 없어 수형을 보기 쉽다.'